# Extraction des connaissances des données

Les données choisies proviennent d'un dataset fournis par audi. Ce sont des données extraite des entrainements de leurs voitures autonome. 
https://www.a2d2.audi/a2d2/en/dataset.html

Il est composé de:
* Données lidar
* Vidéos des caméras embarqués
* Bounding box 3D détectées par leurs véhicule
* Signaux du bus CAN

Nous avons choisis les signaux CAN car ce sont les données les plus simple à analyser et les plus légère (le dataset complet pèse 2.3To).
Les données sont fournis dans un fichier json contenant toutes les informations. Nous allons tout d'abord extraire les données dans un fichier csv simplifier en extrayant et formatant les informations suivantes:
* Pression de la pédale d'accèlération
* Pression de la pédale de freinage
* Distances retournées par les capteurs avant et arrière droite et gauche
* Vitesse du véhicule

In [41]:
import json
import csv
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

## Préparation des données

### Lecture des données brut

In [2]:
with open('bus_signals.json', 'r') as input_file:
    initial_datas = json.load(input_file)
print(initial_datas.keys())

dict_keys(['accelerator_pedal', 'accelerator_pedal_gradient_sign', 'angular_velocity_omega_x', 'angular_velocity_omega_y', 'angular_velocity_omega_z', 'brake_pressure', 'distance_pulse_front_left', 'distance_pulse_front_right', 'distance_pulse_rear_left', 'distance_pulse_rear_right', 'latitude_degree', 'latitude_direction', 'longitude_degree', 'longitude_direction', 'pitch_angle', 'roll_angle', 'steering_angle_calculated', 'steering_angle_calculated_sign', 'vehicle_speed'])


In [3]:
keys = ['accelerator_pedal', 'brake_pressure', 'distance_pulse_front_left', 'distance_pulse_front_right', 'distance_pulse_rear_left', 'distance_pulse_rear_right', 'vehicle_speed']
units = [initial_datas[key]['unit'] if not initial_datas[key]['unit'] is None else 'CentiMeter' for key in keys]

### Extraction de formatage des données initiales

Chaque données est fournis avec un timestamp d'une précision de 17 digits. Nous allons diminuer cette précision afin de pouvoir fusionner des données et obtenir des lignes complètes sans donnée manquante.

In [4]:

merged_datas = dict()
for key in keys:
    values = initial_datas[key]['values'].copy()
    for i in range(len(values)):
        timestamp = round(values[i][0]/1e4)
        if not timestamp in merged_datas:
            merged_datas[timestamp] = dict(accelerator_pedal=None, brake_pressure=None, distance_pulse_front_left=None, distance_pulse_front_right=None, distance_pulse_rear_left=None, distance_pulse_rear_right=None, vehicle_speed=None)
        if merged_datas[timestamp][key] is None:
            merged_datas[timestamp][key] = values[i][1]


### Création du fichier csv

Nous sauvegardons les données dans un fichier csv afin de pouvoir les recharger plus facilement, et au besoin les ouvrir dans R

In [5]:
timestamps = sorted(list(merged_datas.keys()))
list_data = []
for timestamp in timestamps:
    row = [timestamp] + [merged_datas[timestamp][key] for key in keys]
    if not None in row:
        list_data.append(row)

In [6]:
with open('bus_signals.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['timestamp'] + keys)
    csv_writer.writerow(['MicroSecond'] + units)
    for i in range(len(list_data)):
        csv_writer.writerow(list_data[i])

## Analyse des données

### Préparations des données

In [3]:
datas = list()
header = None

with open('./bus_signals.csv', 'r') as datas_file:
    csv_reader = csv.reader(datas_file)
    index = 0
    for row in csv_reader:
        if index != 0:
            datas.append([float(col) for col in row])
        else:
            header = row
        index += 1

In [5]:
X = list()
y = list()

for row in datas:
    y.append(row[1:3])
    X.append(row[3:])

[179.0, 84.0, 658.0, 464.0, 10.36]
[38.8, 0.0]


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

### Question 6: Arbre de régression

In [28]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6192562908815413

### Question 7: Neural Network

In [36]:
reg = MLPRegressor(max_iter=500, activation='logistic', learning_rate='adaptive', verbose=True, hidden_layer_sizes=(50,50,))
reg = reg.fit(X_train, y_train)
reg.score(X_test, y_test)

Iteration 1, loss = 54.04416972
Iteration 2, loss = 40.40807755
Iteration 3, loss = 37.05716456
Iteration 4, loss = 36.34970750
Iteration 5, loss = 36.26072155
Iteration 6, loss = 36.25410618
Iteration 7, loss = 36.24370966
Iteration 8, loss = 36.23388865
Iteration 9, loss = 36.21622092
Iteration 10, loss = 36.17212562
Iteration 11, loss = 36.02364794
Iteration 12, loss = 35.48018443
Iteration 13, loss = 34.36625038
Iteration 14, loss = 33.15236260
Iteration 15, loss = 32.50963956
Iteration 16, loss = 31.95665751
Iteration 17, loss = 31.71948822
Iteration 18, loss = 31.42636623
Iteration 19, loss = 31.30429274
Iteration 20, loss = 30.98260580
Iteration 21, loss = 30.93628580
Iteration 22, loss = 30.90169112
Iteration 23, loss = 30.79560852
Iteration 24, loss = 30.70823875
Iteration 25, loss = 30.66400363
Iteration 26, loss = 30.64002126
Iteration 27, loss = 30.57875484
Iteration 28, loss = 30.47219646
Iteration 29, loss = 30.45914793
Iteration 30, loss = 30.39676837
Iteration 31, loss 

0.23277538301867684

In [42]:
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8510858032092434